In [1]:
# a) extend the user input with similar sentences to have more input

# setup:
# 1. given a dataset, and user input (active learning/ random), augment user input with X number of similar instances on confirmation and 
# complaints, respectively. and make the complaint_sim set to be expected label as the opposite and confirm set to be the expected label as
# the same

In [2]:
from rbbm_src.labelling_func_src.src.example_tree_rules import *
from rbbm_src.labelling_func_src.src.LFRepair import run_snorkel
from rbbm_src.labelling_func_src.src.classes import clean_text
from snorkel.utils import probs_to_preds
import psycopg2
import pandas as pd
from snorkel.labeling import (
	LabelingFunction, 
	labeling_function, 
	PandasLFApplier, 
	LFAnalysis,
	filter_unlabeled_dataframe
	)
from snorkel.labeling.model import MajorityLabelVoter, LabelModel

[nltk_data] Downloading package words to /home/opc/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to /home/opc/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2024-05-18 21:28:07.369040: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-18 21:28:07.542357: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-18 21:28:08.427105: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or di

In [3]:
from sentence_transformers import SentenceTransformer, util

In [4]:
conn = psycopg2.connect(dbname='label', user='postgres')

In [5]:
sentences_df=pd.read_sql(f'SELECT * FROM imdb', conn)
sentences_df = sentences_df.rename(columns={"class": "expected_label", "content": "old_text"})
sentences_df['text'] = sentences_df['old_text'].apply(lambda s: clean_text(s))
sentences_df = sentences_df[~sentences_df['text'].isna()]

/tmp/ipykernel_17956/1219498985.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sentences_df=pd.read_sql(f'SELECT * FROM imdb', conn)


In [6]:
# lf_internal_args = lf_input_internal(funcs=LFs)
imdb_rules=gen_imdb_funcs()

lfs = [f.gen_label_rule() for f in imdb_rules]
applier = PandasLFApplier(lfs=lfs)
initial_vectors = applier.apply(df=sentences_df, progress_bar=False)
snorkel_model = LabelModel(cardinality=2, verbose=True, device='cpu')
snorkel_model.fit(L_train=initial_vectors, n_epochs=500, log_freq=100, seed=123)

probs_test= snorkel_model.predict_proba(L=initial_vectors)


INFO 2024-05-18 21:28:32,661 [label_model.py:fit:905] Computing O...
INFO 2024-05-18 21:28:32,669 [label_model.py:fit:911] Estimating \mu...
  0%|                                                                                                                                                                                                                                                                            | 0/500 [00:00<?, ?epoch/s]INFO 2024-05-18 21:28:32,674 [logger.py:log:79] [0 epochs]: TRAIN:[loss=0.039]
INFO 2024-05-18 21:28:32,739 [logger.py:log:79] [100 epochs]: TRAIN:[loss=0.001]
 30%|█████████████████████████████████████████████████████████████████████████████▎                                                                                                                                                                                  | 151/500 [00:00<00:00, 1505.31epoch/s]INFO 2024-05-18 21:28:32,806 [logger.py:log:79] [200 epochs]: TRAIN:[loss=0.000]
INFO 2024-05-18 21

In [7]:
probs_test

array([[5.00000000e-01, 5.00000000e-01],
       [5.00000000e-01, 5.00000000e-01],
       [5.00000000e-01, 5.00000000e-01],
       ...,
       [1.65185586e-01, 8.34814414e-01],
       [3.29956851e-01, 6.70043149e-01],
       [9.99914759e-01, 8.52411414e-05]])

In [8]:
probs_test.shape

(50000, 2)

In [9]:
df_sentences_filtered, probs_test_filtered, filtered_vectors = filter_unlabeled_dataframe(
        X=sentences_df, y=probs_test, L=initial_vectors
)

In [10]:
df_sentences_filtered.shape

(24161, 5)

In [11]:
# model = SentenceTransformer('all-MiniLM-L6-v2')

# # user_sentence_index = 0
# # user_sentence = corpus[user_sentence_index]
# corpus = df_sentences_filtered['text'].to_list()
# corpus_embeddings = model.encode(corpus)


In [12]:
import numpy as np

In [13]:
df_sentences_filtered = df_sentences_filtered.reset_index(drop=True)
prob_diffs = [abs(t[0]-t[1]) for t in probs_test_filtered]
prob_diffs_tuples = [(t[0],t[1]) for t in probs_test_filtered]
df_sentences_filtered['model_pred_diff'] = pd.Series(prob_diffs)
df_sentences_filtered['model_pred_prob_tuple'] = pd.Series(prob_diffs_tuples)
df_sentences_filtered['model_pred'] = pd.Series(snorkel_model.predict(L=filtered_vectors))
cached_vectors = dict(zip(lfs, np.transpose(filtered_vectors)))
df_sentences_filtered['vectors'] = pd.Series([",".join(map(str, t)) for t in filtered_vectors])
# df_sentences_filtered.to_csv('result.csv')
# the wrong labels we get
wrong_preds = df_sentences_filtered[(df_sentences_filtered['expected_label']!=df_sentences_filtered['model_pred'])]
# df_sentences_filtered.to_csv('predictions_shakira.csv', index=False)
wrong_preds['signal_strength'] = wrong_preds['vectors'].apply(lambda s: sum([1 for i in s.split(",") if int(i) == SPAM or int(i)==HAM]))
wrong_preds = wrong_preds.sort_values(['signal_strength'], ascending=False)
# logger.critical(wrong_preds)
accuracy=(len(df_sentences_filtered)-len(wrong_preds))/len(df_sentences_filtered)

print(f"""
    out of {len(sentences_df)} sentences, {len(df_sentences_filtered)} actually got at least one signal to \n
    make prediction. Out of all the valid predictions, we have {len(wrong_preds)} wrong predictions, \n
    accuracy = {(len(df_sentences_filtered)-len(wrong_preds))/len(df_sentences_filtered)} 
    """)


    out of 50000 sentences, 24161 actually got at least one signal to 

    make prediction. Out of all the valid predictions, we have 4360 wrong predictions, 

    accuracy = 0.8195438930507843 
    


/tmp/ipykernel_17956/394497290.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wrong_preds['signal_strength'] = wrong_preds['vectors'].apply(lambda s: sum([1 for i in s.split(",") if int(i) == SPAM or int(i)==HAM]))


In [38]:
df_sentences_filtered = df_sentences_filtered.sample(1000)

In [14]:
all_wrongs=df_sentences_filtered[df_sentences_filtered['expected_label']!=df_sentences_filtered['model_pred']]
all_confirms=df_sentences_filtered[df_sentences_filtered['expected_label']==df_sentences_filtered['model_pred']]

In [15]:
all_wrongs=all_wrongs.sort_values('model_pred_diff')
all_confirms=all_confirms.sort_values('model_pred_diff')

In [16]:
confirm_sample_size = wrong_sample_size = 20
augmented_sentences_cnt = 10

In [17]:
sampled_confirms=all_confirms.head(n=confirm_sample_size)
sampled_wrongs=all_wrongs.head(n=wrong_sample_size)


In [18]:
sampled_wrongs.shape

(20, 9)

In [19]:
sampled_confirms.shape

(20, 9)

In [20]:
similar_wrong_sentences = set()
similar_confirm_sentences = set()

In [21]:
# for i, d in sampled_wrongs.iterrows():
#     s = d['text']
#     sentence_embedding = model.encode(s)
#     similarities = util.pytorch_cos_sim(sentence_embedding, corpus_embeddings)[0]
#     top_results = np.argsort(-similarities)[1:11]
#     # print(f"Top 10 similar sentences to: ****\n{s}\n*******'")
#     for idx in top_results:
#         # print('---------------------------------------')
#         sim_sentence_id = df_sentences_filtered[df_sentences_filtered['text']==corpus[idx]]['cid'].to_list()[0]
#         similar_wrong_sentences.add((sim_sentence_id, similarities[idx].item()))
#         # print(corpus[idx], "(Score: {:.4f})\n".format(similarities[idx]))
#         # print('---------------------------------------')
#     # break

In [22]:
# similar_wrong_sentences_top_10 = sorted(similar_wrong_sentences, key=lambda x: x[1], reverse=True)[:10]

In [23]:
# similar_wrong_sentences_top_10

In [24]:
# top_10_cids = [x[0] for x in similar_wrong_sentences_top_10]

In [25]:
# df_sentences_filtered[df_sentences_filtered['cid'].isin(top_10_cids)]

In [26]:
# len(df_sentences_filtered[df_sentences_filtered['cid'].isin(top_10_cids)][df_sentences_filtered['expected_label']!=df_sentences_filtered['model_pred']])

In [41]:
import concurrent.futures
from transformers import pipeline, AutoTokenizer
from tqdm import tqdm
import torch

# Load a pre-trained sentiment analysis pipeline
device = 0 if torch.cuda.is_available() else -1
sentiment_pipeline = pipeline('sentiment-analysis', model='distilbert-base-uncased-finetuned-sst-2-english', device=device)

# Initialize the tokenizer separately
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased-finetuned-sst-2-english')

# Maximum token length for BERT-based models
max_token_length = 512

# Function to tokenize and truncate text to the maximum token length
def tokenize_and_truncate(text, tokenizer, max_length):
    encoded = tokenizer(text, truncation=True, padding='max_length', max_length=max_length, return_tensors='pt')
    return encoded.input_ids[0]

# List of sentences for sentiment analysis
sentences = df_sentences_filtered['text'].to_list()

# Preprocess sentences: Remove null, empty, and truncate long sentences
cleaned_sentences = [sentence for sentence in sentences if sentence and isinstance(sentence, str)]

# Tokenize and truncate sentences
tokenized_sentences = [tokenize_and_truncate(sentence, tokenizer, max_token_length) for sentence in tqdm(cleaned_sentences, desc="Tokenizing and truncating")]

# Function to convert tokenized inputs back to plain text
def detokenize(tokens, tokenizer):
    return tokenizer.decode(tokens, skip_special_tokens=True)

# Convert tokenized inputs back to plain text
processed_sentences = [detokenize(tokens, tokenizer) for tokens in tokenized_sentences]

# Define a function to analyze sentiment in parallel and maintain references
def analyze_sentiment_parallel(sentences, batch_size=64):
    results = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(sentiment_pipeline, sentences[i:i + batch_size]) for i in range(0, len(sentences), batch_size)]
        for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures), desc="Analyzing Sentiments"):
            results.extend(future.result())
    return results

# Analyze the sentiment of each sentence in parallel
raw_results = analyze_sentiment_parallel(processed_sentences)

# Combine sentences with their sentiment analysis results
results = [{'sentence': sentence, 'label': result['label'], 'score': result['score']}
           for sentence, result in zip(processed_sentences, raw_results)]

# Function to find the top 10 similar sentences based on sentiment scores
def find_top_10_similar_sentences(results, reference_sentence_index):
    reference_score = results[reference_sentence_index]['score']
    similarities = [(result['sentence'], abs(reference_score - result['score'])) for result in results]
    sorted_similarities = sorted(similarities, key=lambda x: x[1])
    return sorted_similarities[:10]

# Example: Find top 10 similar sentences for the first sentence
reference_sentence_index = 0
top_10_similar = find_top_10_similar_sentences(results, reference_sentence_index)

# Print the results
reference_sentence = results[reference_sentence_index]['sentence']
print(f"Reference Sentence: {reference_sentence}\n")
print("Top 10 similar sentences based on sentiment scores:")
for sentence, similarity in top_10_similar:
    print(f"Sentence: {sentence} - Similarity: {similarity:.4f}")



DEBUG 2024-05-19 00:38:34,091 [connectionpool.py:_get_conn:273] Resetting dropped connection: huggingface.co
DEBUG 2024-05-19 00:38:34,146 [connectionpool.py:_make_request:456] https://huggingface.co:443 "HEAD /distilbert-base-uncased-finetuned-sst-2-english/resolve/main/config.json HTTP/1.1" 200 0
DEBUG 2024-05-19 00:38:35,246 [connectionpool.py:_make_request:456] https://huggingface.co:443 "HEAD /distilbert-base-uncased-finetuned-sst-2-english/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
Analyzing Sentiments: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [01:30<00:00,  5.68s/it]

Reference Sentence: okay i know that s clich taken on its surface this is a bad film perhaps in a league with plan from outer space the dialog is suspect but the singlish is quite enjoyable the plot is not quite believable gavin s character overacts excessively while watching the movie somethings happen that truly make you wonder handsome and kim making out on a tank gwen eating a banana in a bath just about everything gavin says and does psssssssssssssssycho these things taken separately are perhaps flaws taken together however they are merely quirks watching this movie with an open mind especially if you re not familiar with singaporean culture and with an open minded group of friends is guaranteed to deliver a lot of laughs and a memorable time you can t go into this movie expecting a masterpiece or even expecting to take anything serious at all if you can take this film for what it is an underdog film about underdogs filled to the brim with its own quirks then you should have a goo

In [42]:
results

[{'sentence': 'okay i know that s clich taken on its surface this is a bad film perhaps in a league with plan from outer space the dialog is suspect but the singlish is quite enjoyable the plot is not quite believable gavin s character overacts excessively while watching the movie somethings happen that truly make you wonder handsome and kim making out on a tank gwen eating a banana in a bath just about everything gavin says and does psssssssssssssssycho these things taken separately are perhaps flaws taken together however they are merely quirks watching this movie with an open mind especially if you re not familiar with singaporean culture and with an open minded group of friends is guaranteed to deliver a lot of laughs and a memorable time you can t go into this movie expecting a masterpiece or even expecting to take anything serious at all if you can take this film for what it is an underdog film about underdogs filled to the brim with its own quirks then you should have a good tim

In [37]:
# Function to extract sentiment scores
def extract_scores(results):
    return [(result['sentence'], result['score']) for result in results]

# Extract scores
scores = extract_scores(results)

# Function to find the top 10 similar sentences based on sentiment scores
def find_top_10_similar_sentences(scores, reference_sentence_index):
    reference_score = scores[reference_sentence_index][1]
    similarities = [(sentence, abs(reference_score - score)) for sentence, score in scores]
    sorted_similarities = sorted(similarities, key=lambda x: x[1])
    return sorted_similarities[:10]

# Example: Find top 10 similar sentences for the first sentence
reference_sentence_index = 0
top_10_similar = find_top_10_similar_sentences(scores, reference_sentence_index)

# Print the results
reference_sentence = scores[reference_sentence_index][0]
print(f"Reference Sentence: {reference_sentence}\n")
print("Top 10 similar sentences based on sentiment scores:")
for sentence, similarity in top_10_similar:
    print(f"Sentence: {sentence} - Similarity: {similarity:.4f}")

KeyError: 'sentence'

In [32]:
len([r for r in results if r['label']=='POSITIVE'])

8148

In [33]:
len([r for r in results if r['label']=='NEGATIVE'])

16013

In [39]:
df_sentences_filtered[df_sentences_filtered['expected_label']==1].shape

(404, 9)

In [40]:
df_sentences_filtered[df_sentences_filtered['expected_label']==0].shape

(596, 9)

In [83]:
sentences

['my fondness for chris rock varies with his moviesi hated him after lethal weapon  but i hated everyone in that movie after iti like him when he is himself and not holding backlike in dogma well this is his best yetwasnt expecting this to be that goodlaughed my arse off the whole time chris rock delivers a sweet wonderful story backed by some of the funniest comedy ive seen in quite some time loved it',
 'tony arzenta a sicilian hit man or professional killer decides to leave the business and his former employers do not agreein terms of content this highly enjoyable action movie doesnt have one in terms of sheer amusement it is funit is very melodramatic violent quite brutal the car chases are notable arzenta is an unpretentious yet very likable filmmuch better than the current hollywoodian trash that gets the same label it comes from delons rather short flirting with the italian b cinema of the   s it carefully uses delons tough guy persona belonging to the gallery of bad ass thugs t

In [81]:
# Function to calculate sentiment similarity
def sentiment_similarity(sentiment1, sentiment2):
    return abs(sentiment1['score'] - sentiment2['score'])

# Analyze the sentiment of each sentence in the corpus
corpus_sentiments = [(sentence, sentiment_pipeline(sentence)[0]) for sentence in sentences]

# Sample sentence to find similar sentiments
sample_sentence = "I'm very happy with this service."
sample_sentiment = sentiment_pipeline(sample_sentence)[0]

# Sort the corpus sentences by sentiment similarity to the sample sentence
sorted_sentences = sorted(corpus_sentiments, key=lambda x: sentiment_similarity(sample_sentiment, x[1]))

# Get the top 10 similar sentiment sentences
top_10_similar_sentences = sorted_sentences[:10]

# Print the results
print(f"Sample sentence: {sample_sentence}")
for sentence, sentiment in top_10_similar_sentences:
    print(f"Sentence: {sentence}")
    print(f"Sentiment: {sentiment['label']} - Score: {sentiment['score']:.4f}\n")

Sample sentence: I'm very happy with this service.
Sentence: I'm feeling okay about the situation.
Sentiment: 3 stars - Score: 0.7886

Sentence: I love this product!
Sentiment: 5 stars - Score: 0.9135

Sentence: This is the worst service ever.
Sentiment: 1 star - Score: 0.9581

